In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
%matplotlib inline
# nltk.download('stopwords')

#### Install missing packages

In [2]:
if 0 == 1:
    import sys
    !conda install --yes --prefix {sys.prefix} s3fs seaborn scikit-learn

In [3]:
def read_s3_bucket(bucket, data_key):
    data_location = 's3://{}/{}'.format(bucket, data_key)

    chunksize = 1000000
    chunk_list = []
    df_chunk = pd.read_csv(data_location, chunksize=chunksize)
    for chunk in df_chunk:
        chunk_list.append(chunk)

    df = pd.concat(chunk_list)
    return df

In [4]:
%%time
if 1 == 1:
    
    bucket = 'cs410-yelp'
    data_key = 'processed_data/cleaned_reviews.csv'

    #df = read_s3_bucket(bucket, data_key)
    df = pd.read_csv('processed_data/cleaned_reviews.csv')
    df = df.drop(labels='Unnamed: 0', axis=1)
    df['review_stars']   = df['review_stars'].astype(int)
    df['sentiment_text'] = df['sentiment_text'].astype(str)

Wall time: 43.2 s


In [5]:
df.head()
len(df.index)

3527902

In [6]:
stval = df.groupby('review_stars').mean()
stval

,business_stars,review_count,useful
review_stars,,,
1,3.374313,507.893546,1.431601
2,3.529216,569.256905,1.338946
3,3.633498,615.480560,1.189830
4,3.790773,633.882706,1.206635
5,3.983838,647.555980,0.896749


In [7]:
stop_words = [line.rstrip('\n') for line in open('config/stopwords.txt', 'r', encoding='utf-8')] 

In [8]:
stopwords = frozenset(stop_words)

In [11]:
%%time
# CLASSIFICATION
df_classes = df[(df['review_stars'] == 1) | (df['review_stars'] == 3) | (df['review_stars'] == 5)]
df_classes = df_classes[(df_classes['useful'] == 1)]
df_classes.head()
print(df_classes.shape)

# Seperate the data set into X and Y for prediction
x = df_classes['sentiment_text']
y = df_classes['review_stars']
print(x.head())
print(y.head())

(473700, 10)
3     horrible-service boyfriend because pass couple...
14    big chinese-mississauga solid good-food. recen...
25    time year definitely-change quality food gone-...
39    family probably-close year. almost certain cha...
41    unfortunately not-choice dimsum mississauga di...
Name: sentiment_text, dtype: object
3     1
14    3
25    1
39    3
41    3
Name: review_stars, dtype: int32
Wall time: 651 ms


In [12]:
def text_process(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords]

In [13]:
%%time
vocab = CountVectorizer(analyzer=text_process,stop_words=stopwords).fit(x)
print(len(vocab.vocabulary_))
r0 = x[0]
print(r0)
vocab0 = vocab.transform([r0])
print(vocab0)

815552


KeyError: 0

#### Vectorization of the whole review set and and checking the sparse matrix:

In [14]:
%%time
x = vocab.transform(x)
#Shape of the matrix:
print("Shape of the sparse matrix: {}".format(x.shape))
#Non-zero occurences:
print("Non-Zero occurences: {}".format(x.nnz))

# DENSITY OF THE MATRIX
density = (x.nnz / (x.shape[0] * x.shape[1])) * 100
print("Density of the matrix: {}".format(density))

Shape of the sparse matrix: (473700, 815552)
Non-Zero occurences: 14210164
Density of the matrix: 0.003678273754455728
Wall time: 27 s


#### Splitting data set into training and testing set:

In [15]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=101)

In [16]:
def print_results(y_true, y_pred, classifier_name):
    print("Confusion Matrix for {}:".format(classifier_name))
    print(confusion_matrix(y_true, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['1-Star', '3-Star', '5-Star']))
    print("\nScore: {}".format(round(accuracy_score(y_true, y_pred)*100, 2)))

### Multinomial Naive Bayes

In [13]:
%%time
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_train, y_train)
predmnb = mnb.predict(x_test)
print_results(y_test, predmnb, "Multinomial Naive Bayes")

Confusion Matrix for Multinomial Naive Bayes:
[[ 246   70   58]
 [  53  251  283]
 [   7   39 1360]]

Classification Report:
              precision    recall  f1-score   support

      1-Star       0.80      0.66      0.72       374
      3-Star       0.70      0.43      0.53       587
      5-Star       0.80      0.97      0.88      1406

    accuracy                           0.78      2367
   macro avg       0.77      0.68      0.71      2367
weighted avg       0.77      0.78      0.77      2367


Score: 78.45
CPU times: user 21.2 ms, sys: 3.84 ms, total: 25.1 ms
Wall time: 23.5 ms


### RandomForestClassifier

In [14]:
%%time
from sklearn.ensemble import RandomForestClassifier
rmfr = RandomForestClassifier()
rmfr.fit(x_train, y_train)
p = rmfr.predict(x_test)
print_results(y_test, p, "Random Forest Classifier")

/anaconda3/envs/project/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Confusion Matrix for Random Forest Classifier:
[[ 188   65  121]
 [  78  155  354]
 [  45   81 1280]]

Classification Report:
              precision    recall  f1-score   support

      1-Star       0.60      0.50      0.55       374
      3-Star       0.51      0.26      0.35       587
      5-Star       0.73      0.91      0.81      1406

    accuracy                           0.69      2367
   macro avg       0.62      0.56      0.57      2367
weighted avg       0.66      0.69      0.65      2367


Score: 68.57
CPU times: user 4.01 s, sys: 25.9 ms, total: 4.03 s
Wall time: 4.04 s


### Decision Tree

In [15]:
%%time
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(x_train,y_train)
p = dt.predict(x_test)
print_results(y_test, p, "Decision Tree")

Confusion Matrix for Decision Tree:
[[ 183   89  102]
 [  89  211  287]
 [  91  158 1157]]

Classification Report:
              precision    recall  f1-score   support

      1-Star       0.50      0.49      0.50       374
      3-Star       0.46      0.36      0.40       587
      5-Star       0.75      0.82      0.78      1406

    accuracy                           0.66      2367
   macro avg       0.57      0.56      0.56      2367
weighted avg       0.64      0.66      0.64      2367


Score: 65.53
CPU times: user 4.41 s, sys: 14 ms, total: 4.43 s
Wall time: 4.44 s


### Support Vector Machines

In [16]:
%%time
from sklearn.svm import SVC
svm = SVC(random_state=101)
svm.fit(x_train,y_train)
p = svm.predict(x_test)
print_results(y_test, p, "SVM")

/anaconda3/envs/project/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Confusion Matrix for SVM:
[[   0    0  374]
 [   0    0  587]
 [   0    0 1406]]

Classification Report:
              precision    recall  f1-score   support

      1-Star       0.00      0.00      0.00       374
      3-Star       0.00      0.00      0.00       587
      5-Star       0.59      1.00      0.75      1406

    accuracy                           0.59      2367
   macro avg       0.20      0.33      0.25      2367
weighted avg       0.35      0.59      0.44      2367


Score: 59.4
CPU times: user 22.5 s, sys: 200 ms, total: 22.7 s
Wall time: 22.8 s


/anaconda3/envs/project/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### K - Nearest Neighbor Classifier

In [17]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(x_train,y_train)
p = knn.predict(x_test)
print_results(y_test, p, "kNN")

Confusion Matrix for kNN:
[[238  17 119]
 [212  48 327]
 [382  26 998]]

Classification Report:
              precision    recall  f1-score   support

      1-Star       0.29      0.64      0.39       374
      3-Star       0.53      0.08      0.14       587
      5-Star       0.69      0.71      0.70      1406

    accuracy                           0.54      2367
   macro avg       0.50      0.48      0.41      2367
weighted avg       0.59      0.54      0.51      2367


Score: 54.25
CPU times: user 717 ms, sys: 169 ms, total: 886 ms
Wall time: 887 ms


### Multilayer Perceptron

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(x_train,y_train)
p = mlp.predict(x_test)
print_results(y_test, p, "Multilayer Perceptron")

In [ ]:
item = 11
pr = df['sentiment_text'][item]
print(pr)
print("\nActual Rating: {}".format(df['review_stars'][item]))
pr_t = vocab.transform([pr])
print("Predicted Rating: {}".format(mlp.predict(pr_t)[0]))